In [1]:
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import albumentations as A
from albumentations.core.composition import OneOf
import glob
import os

In [2]:
# 경로 설정

data_path = '/workspace/#Ub370#Uc774#Ud130/Data'

img_path_train = '/workspace/#Ub370#Uc774#Ud130/Data/img/train/'
img_path_val = '/workspace/#Ub370#Uc774#Ud130/Data/img/val/'

json_path_train = '/workspace/#Ub370#Uc774#Ud130/Data/label/train/'
json_path_val = '/workspace/#Ub370#Uc774#Ud130/Data/label/val/'

segment_path_train = '/workspace/#Ub370#Uc774#Ud130/Data/segmentation/train/'
segment_path_val = '/workspace/#Ub370#Uc774#Ud130/Data/segmentation/val/'


img_path_train_anger = img_path_train + 'anger/'
img_path_val_anger = img_path_val + 'anger/'

img_path_train_happy = img_path_train + 'happy/'
img_path_val_happy = img_path_val + 'happy/'

img_path_train_panic = img_path_train + 'panic/'
img_path_val_panic = img_path_val + 'panic/'

img_path_train_sadness = img_path_train + 'sadness/'
img_path_val_sadness = img_path_val + 'sadness/'

In [3]:
# Boundind Box 좌표 정보가 담긴 csv 파일

df_train_anger = pd.read_csv(json_path_train + 'train_anger_1.csv')
df_val_anger = pd.read_csv (json_path_val + 'val_anger.csv')

df_train_happy = pd.read_csv(json_path_train + 'train_happy_1.csv')
df_val_happy = pd.read_csv (json_path_val + 'val_happy.csv')

df_train_panic = pd.read_csv(json_path_train + 'train_panic_1.csv')
df_val_panic = pd.read_csv (json_path_val + 'val_panic.csv')

df_train_sadness = pd.read_csv(json_path_train + 'train_sadness_1.csv')
df_val_sadness = pd.read_csv (json_path_val + 'val_sadness.csv')


In [4]:
processed_data = []

# output_dir = '/workspace/Data_Augumentation/Albumentation/image/train/happy/'
# output_csv_path = '/workspace/Data_Augumentation/Albumentation/label/train/albumentation_train_happy.csv'
# input_dir = img_path_train_happy
# df = df_train_happy

output_dir = '/workspace/Data_Augumentation/Albumentation/image/train/anger/'
output_csv_path = '/workspace/Data_Augumentation/Albumentation/label/train/albumentation_train_anger.csv'
input_dir = img_path_train_anger
df = df_train_anger

# output_dir = '/workspace/Data_Augumentation/Albumentation/image/train/panic/'
# output_csv_path = '/workspace/Data_Augumentation/Albumentation/label/train/albumentation_train_panic.csv'
# input_dir = img_path_train_panic
# df = df_train_panic

# output_dir = '/workspace/Data_Augumentation/Albumentation/image/train/sadness/'
# output_csv_path = '/workspace/Data_Augumentation/Albumentation/label/train/albumentation_train_sadness.csv'
# input_dir = img_path_train_sadness
# df = df_train_sadness

# 가져온 이미지 파일들 출력
for index, row in df.iterrows():
    # if index == 5:
    #     break
    # 이미지와 바운딩 박스를 정의 (예: [x_min, y_min, x_max, y_max])
    image = cv2.imread(input_dir + row['filename'])
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # 바운딩 박스 좌표 (예시)
    bboxes = [
        [row['annot_A.boxes.minX'], row['annot_A.boxes.minY'], row['annot_A.boxes.maxX'], row['annot_A.boxes.maxY']],
        [row['annot_B.boxes.minX'], row['annot_B.boxes.minY'], row['annot_B.boxes.maxX'], row['annot_B.boxes.maxY']],
        [row['annot_C.boxes.minX'], row['annot_C.boxes.minY'], row['annot_C.boxes.maxX'], row['annot_C.boxes.maxY']],
        [row['avg_box.minX'], row['avg_box.minY'], row['avg_box.maxX'], row['avg_box.maxY']],
            
    ]  # 각 박스는 [x_min, y_min, x_max, y_max]
    category_ids = [1, 2, 3, 4]  # 각 바운딩 박스에 대응하는 카테고리 ID

    # 변환 파이프라인 정의 (이미지 및 바운딩 박스)
    transform = A.Compose([
        A.Resize(640, 640),  # 크기 조정
        A.HorizontalFlip(p=0.5),  # 수평 뒤집기
        A.Rotate(limit=30, p=1.0),  # 30도까지 회전
        A.RandomBrightnessContrast(p=0.2),  # 밝기 및 대비 조정
    ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['category_id']))

    # 변환 수행
    augmented = transform(image=image, bboxes=bboxes, category_id=category_ids)

    # 변환된 이미지 및 바운딩 박스
    augmented_image = augmented['image']

    augmented_bboxes = augmented['bboxes']

    # 바운딩 박스 중 가장 큰 영역만 남기기
    # 면적 계산: (x_max - x_min) * (y_max - y_min)
    def get_largest_bbox(bboxes):
        largest_bbox = None
        largest_area = 0
        for bbox in bboxes:
            x_min, y_min, x_max, y_max = bbox
            area = (x_max - x_min) * (y_max - y_min)
            if area > largest_area:
                largest_area = area
                largest_bbox = bbox
        return largest_bbox

    # 가장 큰 바운딩 박스만 남기기
    largest_bbox = get_largest_bbox(augmented_bboxes)

    # 변환된 바운딩 박스 출력
    print("변환된 바운딩 박스:", largest_bbox)

    # 변환된 이미지 시각화 (바운딩 박스 그리기)

    x_min, y_min, x_max, y_max = largest_bbox
    # cv2.rectangle(augmented_image, (int(x_min), int(y_min)), (int(x_max), int(y_max)), (255, 0, 0), 2)

    # plt.imshow(augmented_image)
    # plt.show()

    # 처리된 이미지 저장
    os.makedirs(output_dir, exist_ok=True)
    output_path = output_dir + f"albumentation_{row['filename']}"
    cv2.imwrite(output_path, augmented_image)
    print(f"Processed and saved: {output_path}")

    # 변경된 데이터 저장
    processed_data.append({
        'filename': f"albumentation_{row['filename']}",
        'maxX': int(x_max),
        'maxY': int(y_max),
        'minX': int(x_min),
        'minY': int(y_min),
        'faceExp_uploader': row['faceExp_uploader'],
    })

# 처리된 데이터 저장
if processed_data:
    processed_df = pd.DataFrame(processed_data)
    os.makedirs(os.path.dirname(output_csv_path), exist_ok=True)
    processed_df.to_csv(output_csv_path, index=False)
    print(f"Processed CSV saved to: {output_csv_path}")
else:
    print("No data processed.")



변환된 바운딩 박스: [181.77297973632812, 144.66688537597656, 359.55596923828125, 433.5517272949219]
Processed and saved: /workspace/Data_Augumentation/Albumentation/image/train/anger/albumentation_6oj439e3fbcc52759fb3093035b7c0ecc55c93543dae63ec1162f5f6bfc6e481l.jpg
변환된 바운딩 박스: [222.23458862304688, 157.49082946777344, 445.85052490234375, 576.8422241210938]
Processed and saved: /workspace/Data_Augumentation/Albumentation/image/train/anger/albumentation_65rsfe402042f34319e10128c1ab9614e2f967690a64a0016f2c49ff33fe7sy3z.jpg
변환된 바운딩 박스: [170.38475036621094, 88.86538696289062, 454.3531188964844, 454.9703674316406]
Processed and saved: /workspace/Data_Augumentation/Albumentation/image/train/anger/albumentation_b1cbe34734870cc11c33334e02bea93ac3a3b061caab62c0df1c6b9c75430tquz.jpg
변환된 바운딩 박스: [347.7204284667969, 195.8344268798828, 640.0, 640.0]
Processed and saved: /workspace/Data_Augumentation/Albumentation/image/train/anger/albumentation_llfycc0aa29599cc63cace3610fdaaad3a99aab2ee38c9edb6c10966765bbwd

Invalid SOS parameters for sequential JPEG


변환된 바운딩 박스: [94.57865142822266, 0.0, 378.9234313964844, 336.9412536621094]
Processed and saved: /workspace/Data_Augumentation/Albumentation/image/train/anger/albumentation_9gctcc26065d2daab14d5dd7a0286e101827aa5cbca602d0ab61dcdd1d461421v.jpg
변환된 바운딩 박스: [326.8638610839844, 161.2935333251953, 467.9222412109375, 475.42938232421875]
Processed and saved: /workspace/Data_Augumentation/Albumentation/image/train/anger/albumentation_8zjlff36db22c2bbce178c0522c8b50cb668245a0902b9919babb9a9b69960r6u.jpg
변환된 바운딩 박스: [73.26567840576172, 93.29930877685547, 335.9671936035156, 406.2087707519531]
Processed and saved: /workspace/Data_Augumentation/Albumentation/image/train/anger/albumentation_c0565b8c0b6e111306595b29ea475033ff16a1095b3f2b5535063c50b375bd9de.jpg
변환된 바운딩 박스: [206.9247589111328, 90.1250228881836, 461.8546142578125, 389.4105529785156]
Processed and saved: /workspace/Data_Augumentation/Albumentation/image/train/anger/albumentation_6iamb8385d1913611ababcc9896fcae139ce5bb3acbef6f4afdd1ba89935

Invalid SOS parameters for sequential JPEG


변환된 바운딩 박스: [219.56927490234375, 240.39312744140625, 363.056396484375, 453.9535217285156]
Processed and saved: /workspace/Data_Augumentation/Albumentation/image/train/anger/albumentation_3p0gd2290afd21a168212f0a39fa951cdc5b2a6aa47b0e20133818ad9ac017hnh.jpg
변환된 바운딩 박스: [228.86349487304688, 102.11040496826172, 456.14886474609375, 459.9335021972656]
Processed and saved: /workspace/Data_Augumentation/Albumentation/image/train/anger/albumentation_r3j2f4c2f4b4d63c10531d5ddcdfdc37bfaa9f5bbfc38c91cd45fcaa5d5f0vpva.jpg
변환된 바운딩 박스: [136.32321166992188, 37.57194519042969, 459.3042907714844, 638.501708984375]
Processed and saved: /workspace/Data_Augumentation/Albumentation/image/train/anger/albumentation_opiz67dea71b7b5ca7220bc8a01d7dbaead3767e1a7bfda2388085b977a85b9sc.jpg
변환된 바운딩 박스: [162.5433349609375, 115.15064239501953, 408.0204772949219, 427.9356689453125]
Processed and saved: /workspace/Data_Augumentation/Albumentation/image/train/anger/albumentation_wydq00a0af4e77f21077be02722d333907f3c5102

...

변환된 바운딩 박스: [237.8137969970703, 92.84354400634766, 520.5844116210938, 458.7167053222656]
Processed and saved: /workspace/Data_Augumentation/Albumentation/image/train/anger/albumentation_adv1731e62331883f029e33caa7e39cc72a8faf1812cc182c003e109084179fjx.jpg
변환된 바운딩 박스: [152.53170776367188, 204.17123413085938, 353.0260009765625, 549.2932739257812]
Processed and saved: /workspace/Data_Augumentation/Albumentation/image/train/anger/albumentation_k82u04ee9066a33954863b7839b96629e28cd50b9de9e58992243b993d6b9eogf.jpg
변환된 바운딩 박스: [184.05499267578125, 170.08554077148438, 369.4736328125, 591.0721435546875]
Processed and saved: /workspace/Data_Augumentation/Albumentation/image/train/anger/albumentation_oe6026a6a215c4e7a7bb223f440c1faae03b5f26a334032d46b6b403610c3zzqz.jpg


Invalid SOS parameters for sequential JPEG


변환된 바운딩 박스: [135.28851318359375, 170.4098358154297, 327.10211181640625, 559.1546630859375]
Processed and saved: /workspace/Data_Augumentation/Albumentation/image/train/anger/albumentation_dwrz366a579e1e043de42974b9afa2a21b399ac3ee0f11a99060064974e41zbqt.jpg
변환된 바운딩 박스: [179.59686279296875, 35.977420806884766, 403.5785217285156, 306.6484375]
Processed and saved: /workspace/Data_Augumentation/Albumentation/image/train/anger/albumentation_uvm2aa31b54a65652a6b617835ba26369444548643c6ee745074461ffee858ix4.jpg
변환된 바운딩 박스: [135.22091674804688, 133.1721649169922, 383.6104736328125, 463.62353515625]
Processed and saved: /workspace/Data_Augumentation/Albumentation/image/train/anger/albumentation_4o0q3f8af03d4e30c72e12e3c211025f6ca9ef74ef4f6328ef69c10e7c8f947e6.jpg
변환된 바운딩 박스: [171.4496612548828, 173.54193115234375, 347.5425109863281, 403.8448791503906]
Processed and saved: /workspace/Data_Augumentation/Albumentation/image/train/anger/albumentation_0jsg06048a28fde3caad404f93dec397e5c579e83c1c2ced

Invalid SOS parameters for sequential JPEG


변환된 바운딩 박스: [213.5255126953125, 131.56944274902344, 435.4228210449219, 517.0934448242188]
Processed and saved: /workspace/Data_Augumentation/Albumentation/image/train/anger/albumentation_1lty698d2a73458ad2d1ead3f97287a86999f2957b2af628ca7120ccd75fdibf6.jpg
변환된 바운딩 박스: [184.4019012451172, 232.6175994873047, 358.9956970214844, 496.433837890625]
Processed and saved: /workspace/Data_Augumentation/Albumentation/image/train/anger/albumentation_yofr4139c2fa4d65e1ef69691000c597f979496229f7f011ac80e99477969o4ab.jpg
변환된 바운딩 박스: [73.81033325195312, 155.0877227783203, 301.09197998046875, 514.1690673828125]
Processed and saved: /workspace/Data_Augumentation/Albumentation/image/train/anger/albumentation_ih23e7770f0fb2eee5713b282a3c17fba4bce7eefb1390e97f1f4cd4311fawaod.jpg
변환된 바운딩 박스: [208.35829162597656, 102.13568878173828, 504.0711669921875, 469.69970703125]
Processed and saved: /workspace/Data_Augumentation/Albumentation/image/train/anger/albumentation_2tby14ba67fea06c15c1c19c15c47b0a7c90bbe3f0607